In [0]:
import pandas as pd
import numpy as np

In [0]:
ratings = open('ratings.dat', 'r').readlines()
users= open('users.dat', 'r').readlines()
movies= open('movies.dat', 'r',encoding="latin-1").readlines()

In [8]:
ratings[:5]

['1::1193::5::978300760\n',
 '1::661::3::978302109\n',
 '1::914::3::978301968\n',
 '1::3408::4::978300275\n',
 '1::2355::5::978824291\n']

In [9]:
users[:5]

['1::F::1::10::48067\n',
 '2::M::56::16::70072\n',
 '3::M::25::15::55117\n',
 '4::M::45::7::02460\n',
 '5::M::25::20::55455\n']

In [10]:
movies[:5]

["1::Toy Story (1995)::Animation|Children's|Comedy\n",
 "2::Jumanji (1995)::Adventure|Children's|Fantasy\n",
 '3::Grumpier Old Men (1995)::Comedy|Romance\n',
 '4::Waiting to Exhale (1995)::Comedy|Drama\n',
 '5::Father of the Bride Part II (1995)::Comedy\n']

In [0]:
ratings_list = [i.strip().split("::") for i in open('ratings.dat', 'r').readlines()]
users_list = [i.strip().split("::") for i in open('users.dat', 'r').readlines()]
movies_list = [i.strip().split("::") for i in open('movies.dat', 'r',encoding="latin-1").readlines()]

In [0]:
ratings = np.array(ratings_list)
users = np.array(users_list)
movies = np.array(movies_list)

In [0]:
ratings_df = pd.DataFrame(ratings_list, columns = ['UserID', 'MovieID', 'Rating', 'Timestamp'], dtype = int)
movies_df = pd.DataFrame(movies_list, columns = ['MovieID', 'Title', 'Genres'])
movies_df['MovieID'] = movies_df['MovieID'].apply(pd.to_numeric)

In [14]:
movies_df.head(5)

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [15]:

ratings_df.head(5)

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [16]:
R_df = ratings_df.pivot(index = 'UserID', columns ='MovieID', values = 'Rating').fillna(0)
R_df.head()

MovieID,1,10,100,1000,1002,1003,1004,1005,1006,1007,1008,1009,101,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,102,1020,1021,1022,1023,1024,1025,1026,1027,1028,1029,103,1030,1031,1032,1033,1034,...,960,961,962,963,964,965,966,967,968,969,97,970,971,972,973,974,975,976,977,978,98,980,981,982,984,985,986,987,988,989,99,990,991,992,993,994,996,997,998,999
UserID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,5,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10,5,0,0,0,0,0,0,0,0,0,0,5,0,0,0,3,0,0,3,5,0,0,4,0,3,0,5,5,0,4,0,0,5,0,0,3,4,4,0,0,...,0,0,0,5,0,0,0,0,0,3,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1000,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1001,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [17]:
R=R_df.values
R



array([['5', 0, 0, ..., 0, 0, 0],
       ['5', 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       ['4', 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=object)

In [18]:
R=R.astype(int)
R
user_ratings_mean = np.mean(R, axis = 1)
user_ratings_mean

array([0.05990286, 0.44522396, 0.06206152, ..., 0.03184026, 0.15002698,
       0.35429034])

In [19]:
user_ratings_mean.reshape(-1, 1) #-1 is for unknown dimension


array([[0.05990286],
       [0.44522396],
       [0.06206152],
       ...,
       [0.03184026],
       [0.15002698],
       [0.35429034]])

In [20]:
R_demeaned = R - user_ratings_mean.reshape(-1, 1)
R

array([[5, 0, 0, ..., 0, 0, 0],
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [0]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_demeaned, k = 50)

In [22]:
U.shape

(6040, 50)

In [0]:
sigma = np.diag(sigma)

In [26]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
all_user_predicted_ratings

array([[ 4.28886061e+00, -1.95949954e-01,  5.97905159e-02, ...,
         3.69448331e-02,  6.05339706e-02,  5.07522018e-02],
       [ 4.51691241e+00,  9.42799641e-01, -1.85102236e-01, ...,
        -1.14662683e-01, -1.53838225e-01, -2.73472961e-01],
       [-7.85461022e-02,  9.07656597e-01, -1.24355693e-02, ...,
         2.40143123e-02, -1.50433388e-03, -2.77038221e-02],
       ...,
       [ 2.51096600e+00, -7.73346045e-03, -9.30083673e-02, ...,
         2.92685658e-02, -1.09956651e-02, -2.10282212e-01],
       [ 1.96157517e+00, -3.36833956e-01,  1.24358241e-02, ...,
         1.70794645e-01,  5.40658314e-02, -1.75652918e-01],
       [ 1.32156042e-01,  2.83102525e+00,  9.24648755e-01, ...,
        -3.50771301e-02,  5.57506501e-01,  4.92852217e-01]])

In [27]:
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = R_df.columns)
preds_df.head()

MovieID,1,10,100,1000,1002,1003,1004,1005,1006,1007,1008,1009,101,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,102,1020,1021,1022,1023,1024,1025,1026,1027,1028,1029,103,1030,1031,1032,1033,1034,...,960,961,962,963,964,965,966,967,968,969,97,970,971,972,973,974,975,976,977,978,98,980,981,982,984,985,986,987,988,989,99,990,991,992,993,994,996,997,998,999
0,4.288861,-0.195950,0.059791,0.013902,-0.005269,0.068531,0.034708,0.051772,0.019438,0.069625,0.034058,0.039443,-0.041830,0.087349,-0.031790,0.658264,0.531724,0.255934,0.393800,0.014146,0.277333,0.089554,-0.010552,-0.032431,0.127067,0.280932,1.957301,0.745504,0.259501,0.770961,0.015924,0.093013,2.330511,1.815968,0.019232,0.565711,0.349816,1.496343,0.832117,0.194071,...,-0.017545,0.019434,-0.019179,0.117120,-0.114923,-0.064284,-0.011705,-0.039491,-0.023836,0.247645,0.028113,-0.089022,0.102647,-0.005469,-0.037965,-0.024201,-0.004611,-0.059884,-0.001455,-0.041727,0.027840,-0.026786,0.000670,0.029744,-0.024126,0.014468,0.492302,0.014222,0.014796,-0.003501,0.015478,0.025567,-0.089656,0.045402,0.045417,0.058002,0.006646,0.036945,0.060534,0.050752
1,4.516912,0.942800,-0.185102,-0.070946,-0.031497,-0.132928,-0.088461,0.403236,-0.046937,0.974084,0.455475,1.483934,0.137807,1.395248,0.500725,1.519884,1.610592,1.110748,1.238701,1.083104,1.535655,0.661972,2.554609,0.100624,1.446844,0.947112,3.242170,1.260565,0.710738,1.989405,0.019522,1.623485,5.555681,2.889683,-0.122643,2.201163,2.052373,3.231087,1.467742,-0.168436,...,0.023250,0.105317,-0.016513,0.551367,0.046760,0.528307,-0.010428,0.050150,0.658843,3.141239,-0.062712,0.055502,0.882810,0.038660,0.182204,-0.008714,-0.032490,0.198081,-0.035103,0.041492,-0.041920,-0.001029,-0.039930,0.348567,0.078950,-0.019579,1.008652,-0.046006,0.028251,-0.029895,-0.207525,-0.057530,0.063368,-0.005378,-0.037052,-0.078082,0.059184,-0.114663,-0.153838,-0.273473
2,-0.078546,0.907657,-0.012436,-0.018223,-0.004993,-0.009990,0.088658,-0.079309,-0.001477,0.027198,0.022625,0.118641,-0.129227,0.006508,0.090372,-0.053784,-0.066508,-0.001307,-0.022807,0.056267,0.132570,0.098557,0.278434,-0.010618,-0.044899,-0.105624,0.059315,-0.023585,-0.000903,-0.177053,-0.013366,0.072811,0.128207,0.027774,0.010279,0.235710,0.235974,0.054489,-0.144097,0.067733,...,0.013770,-0.006269,-0.028552,-0.043960,0.021263,-0.271413,-0.003276,0.012869,-0.045798,1.196269,0.004003,-0.014681,-0.132865,-0.011575,-0.045543,0.012365,-0.007446,-0.024158,-0.008780,-0.049993,-0.005691,0.001846,-0.015212,-0.044794,0.012417,-0.000856,0.018273,0.016554,0.048695,-0.009919,-0.010521,-0.017129,0.103189,-0.007191,0.002086,0.283959,0.051613,0.024014,-0.001504,-0.027704
3,3.800371,0.528449,-0.124235,-0.016057,-0.000831,-0.207614,-0.140680,0.048933,-0.088665,0.111042,0.008149,0.090402,-0.022742,0.304303,0.106017,0.527734,0.356338,0.230571,0.189638,0.153129,0.301454,0.206297,-0.172533,0.028763,0.004456,0.197220,1.712931,0.785663,0.346601,0.951999,0.003179,-0.154866,1.375573,1.746050,-0.047624,0.453618,0.229636,1.295694,0.974143,0.027037,...,0.004715,-0.005883,-0.017889,0.046055,-0.096885,0.051222,-0.018967,-0.015980,0.000785,1.535231,-0.038120,-0.013625,0.309765,-0.007636,-0.075131,-0.005726,0.000585,-0.061634,0.004746,-0.005602,-0.000101,-0.017207,0.006196,0.094171,0.016259,0.008076,0.188946,-0.044121,0.089228,-0.000339,0.062178,-0.091779,-0.176755,-0.025442,-0.033945,-0.159619,-0.151297,-0.029600,0.087014,-0.181658
4,3.058320,0.926521,0.218438,0.169113,0.168728,0.271766,0.031149,0.133045,0.314324,-0.391314,0.176537,-0.373878,0.525824,-0.327357,-0.095607,-0.119007,0.333514,0.365883,0.611988,-0.119166,0.168191,0.251651,-0.971618,0.269406,0.554041,0.230258,0.723967,0.282834,0.167602,0.205414,0.213471,-0.027533,0.084112,0.539347,0.110505,-0.146220,-0.118089,0.003905,0.281460,0.670691,...,0.091108,0.093235,0.211783,-0.003128,0.073976,-0.185828,0.142369,0.084298,0.055347,-0.432223,0.317752,0.093964,-0.080712,0.169827,-0.165698,0.185943,0.128833,0.195590,0.124882,0.190022,0.162754,0.118998,0.075102,0.117573,0.077080

In [0]:
ratings_df.dtypes
ratings_df["MovieID"] = pd.to_numeric(ratings_df["MovieID"])
ratings_df = ratings_df.astype(object)
movies_df=movies_df.astype(object)

In [0]:
def recommend_movies(predictions_df, userID, movies_df, original_ratings_df, num_recommendations=5):
    # Get and sort the user's predictions
    user_row_number = userID - 1 # UserID starts at 1, not 0
    sorted_user_predictions = preds_df.iloc[user_row_number].sort_values(ascending=False) # UserID starts at 1
    
    # Get the user's data and merge in the movie information.
    user_data = original_ratings_df[original_ratings_df.UserID == (userID)]
    user_full = (user_data.merge(movies_df, how = 'left', left_on = 'MovieID', right_on = 'MovieID').
                     sort_values(['Rating'], ascending=False)
                 )

    print('User {0} has already rated {1} movies.'.format(userID, user_full.shape[0]))
    print('Recommending highest {0} predicted ratings movies not already rated.'.format(num_recommendations))
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (movies_df[~movies_df['MovieID'].isin(user_full['MovieID'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'MovieID',
               right_on = 'MovieID').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full, recommendations

In [75]:
already_rated, predictions = recommend_movies(preds_df, 835, movies_df, ratings_df, 5)

User 835 has already rated 0 movies.
Recommending highest 5 predicted ratings movies not already rated.


In [76]:
already_rated.head(10)

,UserID,Rating,Timestamp,MovieID,Title,Genres


In [77]:
predictions

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
